<a href="https://colab.research.google.com/github/pliniodester/IR_Bovespa/blob/main/IR_ativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy  as np
import pandas as pd
import io
import datetime

In [ ]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(io.BytesIO(uploaded['Negociações Bovespa.csv']))
# Dataset is now stored in a Pandas Dataframe
display(df)

In [ ]:
# date in yyyy/mm/dd format (UPDATE EVERY MONTH)
Date0 = datetime.datetime(2010,   1,  1)
Datef = datetime.datetime(2022,  12, 31)

Old_tickers = ['KROT3', 'ESTC3', 'NATU3', 'FFCI11', 'CPTS11B', 'VIVT4', 'TIET11', 'RLOG3']
New_tickers = ['COGN3', 'YDUQ3', 'NTCO3', 'RCRB11', 'CPTS11' , 'VIVT3', 'AESB3' , 'CSAN3']

Split_ticker = ['IRBR3'   , 'UGPA3'   , 'GRND3'   , 'ITUB3'   , 'RADL3'   , 'SAPR11'  , 'CSMG3'   , 'CSAN3'   , 'WEGE3'   , 'SHUL4'   , 'CSAN3'   , 'B3SA3']
Split_exdate = ['26/09/19', '18/04/19', '01/06/18', '21/11/18', '23/09/20', '30/03/20', '30/11/20', '08/03/21', '27/04/21', '24/04/21', '06/05/21', '17/05/21']
Split_ratio  = [3         , 2         , 3         , 1.5       , 5         , 3         , 3         , 0.2536068 , 2         , 2         , 4         , 3]

In [ ]:
df2 = df

df2['Data'] = pd.to_datetime(df2['Data'], dayfirst = True)
df2 = df2.sort_values(by='Data')

df2['Código'] = df2['Código'].str.replace(r'F$', '')

df2['Preço Médio de Compra'] = \
  df2['Preço Médio de Compra'].apply(lambda x: x.replace(',','.')).astype(float)
df2['Preço Médio de Venda'] = \
  df2['Preço Médio de Venda'].apply(lambda x: x.replace(',','.')).astype(float)

df2['Quantidade Líquida'] = (df2['Quantidade de Compra'] - df2['Quantidade de Venda'])
df2.loc[df2['Quantidade Líquida']>0, 'Preço Médio'] = \
  df2.loc[df2['Quantidade Líquida']>0,'Preço Médio de Compra']
df2.loc[df2['Quantidade Líquida']<0, 'Preço Médio'] = \
  df2.loc[df2['Quantidade Líquida']<0,'Preço Médio de Venda']
df2.loc[df2['Quantidade Líquida']==0, 'Preço Médio'] = 0  

df2['Valor Líquido'] = df2['Quantidade Líquida']*df2['Preço Médio']

for ind in range(0,len(Old_tickers)):
  df2['Código'] = \
    df2['Código'].apply(lambda x: x.replace(Old_tickers[ind],New_tickers[ind]))

Split_exdate = pd.to_datetime(Split_exdate, dayfirst = True)
for ind in range(0,len(Split_ticker)):
  aux = (df2['Código']==Split_ticker[ind])&(df2['Data']<Split_exdate[ind])
  df2.loc[aux, 'Quantidade Líquida'] *= Split_ratio[ind]
  df2.loc[aux, 'Preço Médio'] /= Split_ratio[ind]

DayTrade = pd.DataFrame(df2['Código'],columns=['Código'])
DayTrade['Data'] = df2['Data']
DayTrade['Quantidade Líquida'] = \
  df2[['Quantidade de Compra','Quantidade de Venda']].min(axis=1).astype(int)
DayTrade = DayTrade[DayTrade['Quantidade Líquida']!=0]
DayTrade['Lucro'] = (df2['Preço Médio de Venda']-df2['Preço Médio de Compra'])* \
  DayTrade['Quantidade Líquida']
display(DayTrade)

df2 = df2.drop(['Quantidade de Compra', 'Quantidade de Venda',\
                'Preço Médio de Compra','Preço Médio de Venda','Posição'], axis=1)
display(df2)

In [ ]:
# This block calc average price, stock profits, day trade and FIIs trade
data = [[0,0,0,0]] 
df2['Lucro'] = 0.00
for ticker in np.unique(df2['Código']):
  stocks = 0
  mean = 0.0
  profit = 0
  for ind in df2[(df2['Código'] == ticker) & (df2['Data'] <= Datef)].index:
    if df2['Valor Líquido'][ind] > 0:
      mean = (mean*stocks + \
              df2['Valor Líquido'][ind])/(stocks + df2['Quantidade Líquida'][ind])
      if stocks + df2['Quantidade Líquida'][ind] <= 0:
        print('Error:',ticker, ind)
    else:
      df2.loc[ind,'Lucro'] = (df2['Preço Médio'][ind]-mean)*(-df2['Quantidade Líquida'][ind])
      if df2['Data'][ind] >= Date0:
        profit += df2['Lucro'][ind]
    stocks += df2['Quantidade Líquida'][ind]
  data = np.append(data,[[ticker, stocks, "{:.2f}".format(mean), \
                          "{:.2f}".format(profit)]],axis=0)
  # print(ticker, stocks, 'x', "{:.2f}".format(mean))
IR = pd.DataFrame(data, columns = ['Ticker', 'Ações','Preço médio','Lucro'])
IR['Ações'] = IR['Ações'].astype(float).astype(int)
IR['Lucro'] = IR['Lucro'].astype(float)

print('****** Preço Médio ******')
print(IR[IR['Ações']>0].drop(['Lucro'],axis=1).to_string())
print('\n****** Swing Trade ******')
print(IR[IR['Lucro']!=0].drop(['Preço médio','Ações'],axis=1).to_string(),'\n')
print('Lucro total = ', "{:.2f}".format(IR['Lucro'].astype(float).sum()))
print('\n****** FIIs Trade (swing trade) ******\n', \
      df2[(df2['Código'].str.endswith('11'))&(df2['Código']!='SAPR11')& \
          (df2['Lucro']!=0)&(df2['Código']!='KLBN11')&(df2['Código']!='TAEE11') & \
          (df2['Código']!='TIET11')&(df2['Código']!='BPAC11')&(df2['Código']!='ALUP11')].to_string(),'\n')
print('\n****** Day Trade ******\n',DayTrade.to_string())

In [ ]:
print(IR[(IR['Ticker'].str.endswith('11')) & (IR['Lucro']!=0)].to_string()) # Print the IR info relative to tickers ending in 11, and profit != 0

In [ ]:
df2 = df2[df2['Data']<'2019-12-01'] # Filter considering date

In [ ]:
df2[df2['Código']=='MULT3'] # Print all trades relative to ticker

In [ ]:
print(df2[(df2['Preço Médio']>60) & (df2['Código']!='')].to_string()) # Filter considering avg price and ticker

In [ ]:
print(df2.to_string()) # Print all trades

In [ ]:
print(IR[(IR['Lucro']!=0)].to_string()) # Print IR the info relative to tickers with profit != 0